In [54]:
using CSV, DataFrames, CategoricalArrays, Plots, Statistics, Random, StatsPlots, Gurobi, JuMP, StatsBase, Random

In [55]:
df = CSV.read("/Users/alessandromorosini/Desktop/MLOpt/Project/banrkupcy_dataset.csv", DataFrame);

In [56]:
train_df = filter(row -> row.year in [1, 2, 3], df)
test_df = filter(row -> row.year in [4, 5], df);

In [57]:
shuffled_indices = shuffle(1:nrow(train_df))
train_df = train_df[shuffled_indices, :]
# train_df = train_df[1:2000, :]; # to be removed for final, this is jsut for tesitng

Row,year,net profit / total assets,total liabilities / total assets,working capital / total assets,current assets / short-term liabilities,[(cash + short-term securities + receivables - short-term liabilities) / (operating expenses - depreciation)] * 365,retained earnings / total assets,EBIT / total assets,book value of equity / total liabilities,sales / total assets,equity / total assets,(gross profit + extraordinary items + financial expenses) / total assets,gross profit / short-term liabilities,(gross profit + depreciation) / sales,(gross profit + interest) / total assets,(total liabilities * 365) / (gross profit + depreciation),(gross profit + depreciation) / total liabilities,total assets / total liabilities,gross profit / total assets,gross profit / sales,(inventory * 365) / sales,sales (n) / sales (n-1),profit on operating activities / total assets,net profit / sales,gross profit (in 3 years) / total assets,(equity - share capital) / total assets,(net profit + depreciation) / total liabilities,profit on operating activities / financial expenses,working capital / fixed assets,logarithm of total assets,(total liabilities - cash) / sales,(gross profit + interest) / sales,(current liabilities * 365) / cost of products sold,operating expenses / short-term liabilities,operating expenses / total liabilities,profit on sales / total assets,total sales / total assets,(current assets - inventories) / long-term liabilities,constant capital / total assets,profit on sales / sales,(current assets - inventory - receivables) / short-term liabilities,total liabilities / ((profit on operating activities + depreciation) * (12/365)),profit on operating activities / sales,rotation receivables + inventory turnover in days,(receivables * 365) / sales,net profit / inventory,(current assets - inventory) / short-term liabilities,(inventory * 365) / cost of products sold,EBITDA (profit on operating activities - depreciation) / total assets,EBITDA (profit on operating activities - depreciation) / sales,current assets / total liabilities,short-term liabilities / total assets,(short-term liabilities * 365) / cost of products sold,equity / fixed assets,constant capital / fixed assets,working capital,(sales - cost of products sold) / sales,(current assets - inventory - short-term liabilities) / (sales - gross profit - depreciation),total costs / total sales,long-term liabilities / equity,sales / inventory,sales / receivables,(short-term liabilities * 365) / sales,sales / short-term liabilities,sales / fixed assets,bankruptcy
,Int64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Int64
1,1,0.40191,0.46823,0.41036,1.92,59.112,0.000598,0.48711,1.1357,2.9855,0.53177,0.5425,1.0921,0.17032,0.48711,336.1,1.086,2.1357,0.48711,0.16316,1.374,1.2933,0.53846,0.13462,0.62568,0.51337,0.90404,9.7214,2.8573,3.4341,0.12192,0.16316,67.248,5.4277,5.1702,0.56462,2.9855,missing,0.53177,0.18912,0.24524,0.027879,0.18036,91.327,89.953,35.763,1.8948,1.6944,0.51707,0.1732,1.829,0.44602,0.18424,3.7027,3.7027,1115.0,0.18912,0.7558,0.83833,0.0,265.65,4.0577,54.53,6.6936,20.788,0
2,1,0.13096,0.8639,0.22298,1.2923,-19.741,0.13096,0.169,0.15754,1.0328,0.1361,0.169,0.2215,0.028674,0.169,1841.1,0.19825,1.1575,0.169,0.028296,32.394,missing,0.18349,0.021925,0.169,0.1361,0.1542,0.31726,15.899,3.9961,0.13601,0.028296,48.154,7.5798,0.21239,0.18349,5.9731,4.5179,0.23701,0.03072,0.080548,0.15291,0.03072,56.498,24.104,0.24704,0.5975,33.455,0.18122,0.030342,1.1413,0.76299,0.13

In [58]:
function impute_missing_with_median!(df)
    for col in names(df)
        # Check if the column has missing values
        if eltype(df[:, col]) <: Union{Missing, Number}
            # Compute the median while skipping missing values
            col_median = median(skipmissing(df[:, col]))
            # Replace missing values with the median
            df[:, col] = coalesce.(df[:, col], col_median)
        end
    end
end

impute_missing_with_median!(train_df)

# Undersampling 

## Random 

In [59]:
function balance_data(data::DataFrame)
    balanced_train_df = DataFrame()  # Initialize an empty DataFrame
    years = unique(data.year)  # Get unique years

    for year in years
        # Filter data for the current year
        year_data = filter(row -> row.year == year, data)
        
        # Separate bankruptcy and non-bankruptcy cases
        bankruptcy = filter(row -> row.bankruptcy == 1, year_data)
        non_bankruptcy = filter(row -> row.bankruptcy == 0, year_data)
        
        n_bankruptcy = size(bankruptcy, 1)  # Count bankruptcy cases
        
        if n_bankruptcy == 0
            println("Year $year has no bankruptcy cases. Skipping under-sampling.")
            continue
        end
        
        # Randomly sample non-bankruptcy cases to match the number of bankruptcy cases
        non_bankruptcy_sampled = non_bankruptcy[rand(1:size(non_bankruptcy, 1), n_bankruptcy), :]
        
        # Combine the sampled data for the current year
        balanced_year_df = vcat(bankruptcy, non_bankruptcy_sampled)
        
        # Append to the overall balanced DataFrame
        balanced_train_df = vcat(balanced_train_df, balanced_year_df)
    end

    # Shuffle the final DataFrame
    shuffled_train_df = balanced_train_df[randperm(size(balanced_train_df, 1)), :]
    return shuffled_train_df
end

balance_data (generic function with 1 method)

In [60]:
random_train_df = balance_data(train_df)
print(size(random_train_df))
first(random_train_df, 3)

(2332, 66)

Row,year,net profit / total assets,total liabilities / total assets,working capital / total assets,current assets / short-term liabilities,[(cash + short-term securities + receivables - short-term liabilities) / (operating expenses - depreciation)] * 365,retained earnings / total assets,EBIT / total assets,book value of equity / total liabilities,sales / total assets,equity / total assets,(gross profit + extraordinary items + financial expenses) / total assets,gross profit / short-term liabilities,(gross profit + depreciation) / sales,(gross profit + interest) / total assets,(total liabilities * 365) / (gross profit + depreciation),(gross profit + depreciation) / total liabilities,total assets / total liabilities,gross profit / total assets,gross profit / sales,(inventory * 365) / sales,sales (n) / sales (n-1),profit on operating activities / total assets,net profit / sales,gross profit (in 3 years) / total assets,(equity - share capital) / total assets,(net profit + depreciation) / total liabilities,profit on operating activities / financial expenses,working capital / fixed assets,logarithm of total assets,(total liabilities - cash) / sales,(gross profit + interest) / sales,(current liabilities * 365) / cost of products sold,operating expenses / short-term liabilities,operating expenses / total liabilities,profit on sales / total assets,total sales / total assets,(current assets - inventories) / long-term liabilities,constant capital / total assets,profit on sales / sales,(current assets - inventory - receivables) / short-term liabilities,total liabilities / ((profit on operating activities + depreciation) * (12/365)),profit on operating activities / sales,rotation receivables + inventory turnover in days,(receivables * 365) / sales,net profit / inventory,(current assets - inventory) / short-term liabilities,(inventory * 365) / cost of products sold,EBITDA (profit on operating activities - depreciation) / total assets,EBITDA (profit on operating activities - depreciation) / sales,current assets / total liabilities,short-term liabilities / total assets,(short-term liabilities * 365) / cost of products sold,equity / fixed assets,constant capital / fixed assets,working capital,(sales - cost of products sold) / sales,(current assets - inventory - short-term liabilities) / (sales - gross profit - depreciation),total costs / total sales,long-term liabilities / equity,sales / inventory,sales / receivables,(short-term liabilities * 365) / sales,sales / short-term liabilities,sales / fixed assets,bankruptcy
,Int64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Int64
1,1,0.12227,0.4406,0.1661,1.5647,-2.8932,0.18976,0.15187,1.2248,0.99931,0.53963,0.15187,0.51628,0.097813,0.15187,754.15,0.48399,2.2696,0.15187,0.06966,29.391,1.3354,0.15933,0.056084,0.23732,0.53963,0.41681,0.73034,0.30773,4.6314,0.15464,0.06966,49.214,7.4165,0.36163,0.15933,2.3674,1.9441,0.68607,0.073084,0.37756,0.065632,0.073084,58.462,29.071,0.69649,0.96786,29.371,0.097955,0.044931,1.0446,0.29415,0.13483,0.99977,1.2711,7107.6,-0.000686,0.22658,1.0007,0.27138,12.419,12.555,49.248,7.4115,4.0391,0
2,3,-0.35682,0.85875,0.064281,1.0922,-135.31,0.0,-0.33528,0.16449,1.2449,0.14125,-0.27216,-0.48112,-0.23612,-0.33528,-1066.3,-0.3423,1.1645,-0.33528,-0.26932,62.317,1.0257,-0.29922,-0.28662,-0.33528,0.09752,-0.36739,-4.7408,0.26913,3.5353,0.65647,-0.25532,237.3,1.5382,1.2482,0.173,1.2449,3.3891,0.30313,0.13897,0.40264,-0.111,-0.24035,140.9,78.581,-1.6788,0.78724

## Optimization

In [61]:
function compute_dist(row1::DataFrameRow, row2::DataFrameRow, features::Vector{Symbol})
    dist = 0.0
    for f in features
        diff = row1[f] - row2[f]
        dist += diff * diff
    end
    return sqrt(dist)
end

# Compute distance matrix between two DataFrames
function compute_dist_matrix(df_bankrupt::DataFrame, df_non_bankrupt::DataFrame, features::Vector{Symbol})
    n_bankrupt = nrow(df_bankrupt)
    n_non_bankrupt = nrow(df_non_bankrupt)
    distances = zeros(Float64, n_bankrupt, n_non_bankrupt)
    
    @info "Computing distance matrix for year $(df_bankrupt.year[1])..."
    
    for i in 1:n_bankrupt
        for j in 1:n_non_bankrupt
            distances[i, j] = compute_dist(df_bankrupt[i, :], df_non_bankrupt[j, :], features)
        end
    end
    
    return distances
end

compute_dist_matrix (generic function with 1 method)

In [62]:

function optimal_matching(dist_matrix::Matrix{Float64})
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)  # Suppress Gurobi output
    
    n_rows, n_cols = size(dist_matrix)
    
    @variable(model, z[1:n_rows, 1:n_cols], Bin)
    
    # Each bankrupt firm is matched to exactly one non-bankrupt firm
    @constraint(model, [i=1:n_rows], sum(z[i, j] for j in 1:n_cols) == 1)
    
    # Each non-bankrupt firm can be matched to at most one bankrupt firm
    @constraint(model, [j=1:n_cols], sum(z[i, j] for i in 1:n_rows) <= 1)
    
    # Objective: minimize total distance
    @objective(model, Min, sum(dist_matrix[i, j] * z[i, j] for i in 1:n_rows, j in 1:n_cols))
    
    optimize!(model)
    
    if termination_status(model) == MOI.OPTIMAL
        assignment = value.(z)
        total_distance = objective_value(model)
        @info "Optimization successful. Total distance: $total_distance"
        return total_distance, assignment
    else
        error("Optimization did not find an optimal solution.")
    end
end

optimal_matching (generic function with 1 method)

In [63]:
function normalize_features!(df::DataFrame, features::Vector{Symbol})
    for f in features
        min_val = minimum(df[!, f])
        max_val = maximum(df[!, f])
        if max_val - min_val != 0
            df[!, f] = (df[!, f] .- min_val) ./ (max_val - min_val)
        else
            # If all values are the same, set to zero to avoid division by zero
            df[!, f] .= 0.0
            @warn "Feature $f has the same value for all records. Normalized to 0."
        end
    end
end

normalize_features! (generic function with 1 method)

In [64]:
feature_cols = names(train_df, Not(["year", "bankruptcy"]))

# Convert feature names to Symbols
feature_symbols = Symbol.(feature_cols)

# Normalize features for the entire dataset first to ensure consistency across years
normalize_features!(train_df, feature_symbols)

# Initialize an empty DataFrame to store the sampled data
sampled_data = DataFrame()

# Get unique years
years = unique(train_df.year)

println("\nStarting under-sampling process...\n")

for year in years
    println("Processing Year: $year")
    
    # Subset data for the current year
    df_year = filter(row -> row.year == year, train_df)
    
    # Separate bankrupt and non-bankrupt firms
    df_bankrupt = filter(row -> row.bankruptcy == 1, df_year)
    df_non_bankrupt = filter(row -> row.bankruptcy == 0, df_year)
    
    n_bankrupt = nrow(df_bankrupt)
    n_non_bankrupt = nrow(df_non_bankrupt)
    
    println("  Number of Bankrupt Firms: $n_bankrupt")
    println("  Number of Non-Bankrupt Firms: $n_non_bankrupt")
    
    if n_bankrupt == 0
        println("  -> No bankrupt firms in year $year. Skipping this year.\n")
        continue
    elseif n_non_bankrupt < n_bankrupt
        println("  -> Not enough non-bankrupt firms to match in year $year.")
        println("     Consider alternative strategies like oversampling or adjusting matching criteria.\n")
        continue
    end
    
    # Compute the distance matrix
    dist_matrix = compute_dist_matrix(df_bankrupt, df_non_bankrupt, feature_symbols)
    
    # Perform optimal matching
    total_dist, assignment = optimal_matching(dist_matrix)
    
    # Extract matched non-bankrupt firms based on the assignment matrix
    matched_indices = Vector{Int}()
    for i in 1:nrow(df_bankrupt)
        for j in 1:nrow(df_non_bankrupt)
            if assignment[i, j] == 1
                push!(matched_indices, j)
                break  # Move to the next bankrupt firm once a match is found
            end
        end
    end
    
    df_matched_non_bankrupt = df_non_bankrupt[matched_indices, :]
    
    # Combine bankrupt and matched non-bankrupt firms
    sampled_year = vcat(df_bankrupt, df_matched_non_bankrupt)
    
    # Append to the sampled_data DataFrame
    append!(sampled_data, sampled_year)
    
    println("  -> Matched $n_bankrupt bankrupt firms with $n_bankrupt non-bankrupt firms.\n")
end

println("Under-sampling process completed.\n")


Starting under-sampling process...

Processing Year: 1
  Number of Bankrupt Firms: 271
  Number of Non-Bankrupt Firms: 6756


[ Info: Computing distance matrix for year 1...
[ Info: Optimization successful. Total distance: 2.6106130228812012


Set parameter Username
Error: invalid user locale; possible fix is to set the system environment
       variable 'LC_ALL' to a valid locale (e.g., to 'C')
Academic license - for non-commercial use only - expires 2025-08-21
  -> Matched 271 bankrupt firms with 271 non-bankrupt firms.

Processing Year: 2
  Number of Bankrupt Firms: 400
  Number of Non-Bankrupt Firms: 9773


[ Info: Computing distance matrix for year 2...
[ Info: Optimization successful. Total distance: 1.8133957148278954


Set parameter Username
Error: invalid user locale; possible fix is to set the system environment
       variable 'LC_ALL' to a valid locale (e.g., to 'C')
Academic license - for non-commercial use only - expires 2025-08-21
  -> Matched 400 bankrupt firms with 400 non-bankrupt firms.

Processing Year: 3
  Number of Bankrupt Firms: 495
  Number of Non-Bankrupt Firms: 10008


[ Info: Computing distance matrix for year 3...
[ Info: Optimization successful. Total distance: 1.2830245049072135


Set parameter Username
Error: invalid user locale; possible fix is to set the system environment
       variable 'LC_ALL' to a valid locale (e.g., to 'C')
Academic license - for non-commercial use only - expires 2025-08-21
  -> Matched 495 bankrupt firms with 495 non-bankrupt firms.

Under-sampling process completed.



In [65]:
println("Verifying the class balance for each year in the sampled dataset:\n")

for year in unique(sampled_data.year)
    df_sampled_year = filter(row -> row.year == year, sampled_data)
    counts = combine(groupby(df_sampled_year, :bankruptcy), nrow => :count)
    println("Year $year:")
    for row in eachrow(counts)
        status = row.bankruptcy == 1 ? "Bankrupt" : "Non-Bankrupt"
        println("  $status: $(row.count)")
    end
    println()
end

Verifying the class balance for each year in the sampled dataset:

Year 1:
  Non-Bankrupt: 271
  Bankrupt: 271

Year 2:
  Non-Bankrupt: 400
  Bankrupt: 400

Year 3:
  Non-Bankrupt: 495
  Bankrupt: 495



In [66]:
print(size(sampled_data))
first(sampled_data, 3)

(2332, 66)

Row,year,net profit / total assets,total liabilities / total assets,working capital / total assets,current assets / short-term liabilities,[(cash + short-term securities + receivables - short-term liabilities) / (operating expenses - depreciation)] * 365,retained earnings / total assets,EBIT / total assets,book value of equity / total liabilities,sales / total assets,equity / total assets,(gross profit + extraordinary items + financial expenses) / total assets,gross profit / short-term liabilities,(gross profit + depreciation) / sales,(gross profit + interest) / total assets,(total liabilities * 365) / (gross profit + depreciation),(gross profit + depreciation) / total liabilities,total assets / total liabilities,gross profit / total assets,gross profit / sales,(inventory * 365) / sales,sales (n) / sales (n-1),profit on operating activities / total assets,net profit / sales,gross profit (in 3 years) / total assets,(equity - share capital) / total assets,(net profit + depreciation) / total liabilities,profit on operating activities / financial expenses,working capital / fixed assets,logarithm of total assets,(total liabilities - cash) / sales,(gross profit + interest) / sales,(current liabilities * 365) / cost of products sold,operating expenses / short-term liabilities,operating expenses / total liabilities,profit on sales / total assets,total sales / total assets,(current assets - inventories) / long-term liabilities,constant capital / total assets,profit on sales / sales,(current assets - inventory - receivables) / short-term liabilities,total liabilities / ((profit on operating activities + depreciation) * (12/365)),profit on operating activities / sales,rotation receivables + inventory turnover in days,(receivables * 365) / sales,net profit / inventory,(current assets - inventory) / short-term liabilities,(inventory * 365) / cost of products sold,EBITDA (profit on operating activities - depreciation) / total assets,EBITDA (profit on operating activities - depreciation) / sales,current assets / total liabilities,short-term liabilities / total assets,(short-term liabilities * 365) / cost of products sold,equity / fixed assets,constant capital / fixed assets,working capital,(sales - cost of products sold) / sales,(current assets - inventory - short-term liabilities) / (sales - gross profit - depreciation),total costs / total sales,long-term liabilities / equity,sales / inventory,sales / receivables,(short-term liabilities * 365) / sales,sales / short-term liabilities,sales / fixed assets,bankruptcy
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,1,0.731432,0.1315,0.964238,1.43941e-5,0.92313,0.626059,0.225957,0.00265344,0.000401898,0.304124,0.21749,0.157471,0.0900466,0.225957,0.355214,0.0253417,4.03684e-5,0.225957,0.125576,1.56987e-6,0.0424573,0.241313,0.125576,0.185592,0.27032,0.0253323,0.0431814,0.276261,0.445734,0.039896,0.125412,0.000536874,0.00120127,0.0719481,0.212709,0.00028382,0.00132355,0.304166,0.392921,0.0204739,0.00425431,0.392893,0.00159544,0.00214538,0.692082,0.00190149,1.77639e-5,0.259213,0.980509,9.71984e-6,0.000805015,0.000301773,0.0207822,0.0207827,0.29145,0.999736,0.751021,0.000180154,0.0135695,8.13552e-6,0.000825101,0.0855407,0.000313033,0.0349679,1
2,1,0.731473,0.131058,0.964559,2.35248e-5,0.92313,0.626059,0.22597,0.00267765,0.000239123,0.304279,0.217501,0.157472,0.0900486,0.22597,0.355066,0.0253507,6.4636e-5,0.22597,0.125575,4.97342e-6,0.0424573,0.241308,0.125575,0.185603,0.270481,0.0253422,0.0431813,0.276277,0.527782,0.0398965,0.12541

In [67]:
CSV.write("sampled_data_random.csv", random_train_df)
CSV.write("sampled_data_ROAD.csv", sampled_data)

"sampled_data_ROAD.csv"